In this file, I'll try to analize the video with hidden epilepsy hazard - police flshing lights.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import cv2 as cv

import downloader as d 
import video as v

from typing import Tuple, Union
from numpy.typing import NDArray
from dataclasses import dataclass
from collections import deque

from importlib import reload

In [2]:
def get_video(path: str) -> Tuple[NDArray, int]:
    vid = cv.VideoCapture(path)
    fps = vid.get(cv.CAP_PROP_FPS)
    #video = np.fromiter(vid, np.ndarray)
    frames = []
    ret = True
    while ret:
        ret, img = vid.read() # read one frame from the 'capture' object; img is (H, W, C)
        if ret:
            frames.append(img)
    # dimensions (T, H, W, C)
    return np.stack(frames, axis=0), int(round(fps))

def get_vid_df(path: str) -> pd.DataFrame:
    vid,fps = get_video(path)
    frames = vid.shape[0]
    height = vid.shape[1]
    width = vid.shape[2]
    df = pd.DataFrame(vid.reshape((-1,3)))
    df['frame'] = df.index // (width * height)
    df['x'] = df.index % width
    df['y'] = df.index // width % height
    df['second'] = df.frame // fps
    df = df.set_index(['frame','y','x']).rename(columns={
        0:'hue',
        1:'lightness',
        2:'saturation',
    })
    df.attrs['fps'] = fps
    return df

In [3]:
path = 'videos/Car flips over bridge following police pursuit.mp4'

In [4]:
df = get_vid_df(path)